In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [16]:
# Convert categorical data to numeric and separate target feature for training data
loan_risk_dict = {'low_risk': 0, 'high_risk': 1}
train_df['loan_status'] = train_df['loan_status'].replace(loan_risk_dict)

# train_df.head()

X_train = train_df.drop('loan_status', axis=1)
y_train = train_df['loan_status']
X_train = pd.get_dummies(X_train, drop_first=True)

X_train = X_train.drop('Unnamed: 0', axis=1)
# X_train.head()

columns = list(X_train.columns)

In [20]:
# Convert categorical data to numeric and separate target feature for testing data
test_df = test_df.replace({'loan_status':loan_risk_dict})
X_test = test_df.drop('loan_status', axis=1)
y_test = test_df['loan_status']
X_test = pd.get_dummies(X_test, drop_first=True)
X_test = X_test.drop('Unnamed: 0', axis=1)

In [23]:
# add missing dummy variables to testing set
X_test = X_test.reindex(labels=columns,axis=1)
X_test = X_test.fillna(0)
X_test.head()

,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App,hardship_flag_Y,debt_settlement_flag_Y
0,67991,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,...,99475.0,1,0,0,0,0,1,0,0,0.0
1,25429,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,...,23628.0,0,0,1,0,0,1,0,0,0.0
2,38496,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,...,15000.0,0,0,1,0,0,1,0,0,0.0
3,19667,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,...,35981.0,0,0,1,0,0,1,0,0,0.0
4,37505,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,...,24977.0,0,0,1,0,0,1,0,0,0.0


In [25]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

classifier = LogisticRegression()
classifier

LogisticRegression()

In [ ]:
# Train a Random Forest Classifier model and print the model score

In [ ]:
# Scale the data

In [ ]:
# Train the Logistic Regression model on the scaled data and print the model score

In [ ]:
# Train a Random Forest Classifier model on the scaled data and print the model score